In [3]:
!pip install langgraph==0.3 langchain_openai langchain_core



Defaulting to user installation because normal site-packages is not writeable


In [4]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from typing import List

# ✅ 상태 타입 정의
class CodeState(TypedDict):
    code: str
    style_feedback: str
    bug_feedback: str
    optimize_feedback: str
    improved_code: str

# ✅ LLM 준비
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ✅ 각 노드 정의

def check_style(state: CodeState) -> CodeState:
    prompt = f"""다음 코드의 스타일을 분석하고 피드백을 제공해줘:\n\n```python\n{state['code']}\n```"""
    response = llm.invoke(prompt)
    state["style_feedback"] = response.content
    return state

def check_bug(state: CodeState) -> CodeState:
    prompt = f"""다음 코드에서 버그 가능성이나 주의해야 할 점을 지적해줘:\n\n```python\n{state['code']}\n```"""
    response = llm.invoke(prompt)
    state["bug_feedback"] = response.content
    return state

def check_optimization(state: CodeState) -> CodeState:
    prompt = f"""다음 코드의 성능 또는 최적화 가능성에 대해 분석해줘:\n\n```python\n{state['code']}\n```"""
    response = llm.invoke(prompt)
    state["optimize_feedback"] = response.content
    return state

def improve_code(state: CodeState) -> CodeState:
    prompt = f"""
다음은 코드에 대한 피드백입니다:

[스타일 분석]
{state['style_feedback']}

[버그 분석]
{state['bug_feedback']}

[최적화 분석]
{state['optimize_feedback']}

위 내용을 모두 반영해서 개선된 코드를 작성해줘:\n\n```python\n{state['code']}\n```
"""
    response = llm.invoke(prompt)
    state["improved_code"] = response.content
    return state

# ✅ 그래프 생성
workflow = StateGraph(CodeState)

# 노드 추가
workflow.add_node("스타일 분석", check_style)
workflow.add_node("버그 분석", check_bug)
workflow.add_node("최적화 분석", check_optimization)
workflow.add_node("개선 코드 생성", improve_code)

# 연결 구성
workflow.add_edge(START, "스타일 분석")
workflow.add_edge("스타일 분석", "버그 분석")
workflow.add_edge("버그 분석", "최적화 분석")
workflow.add_edge("최적화 분석", "개선 코드 생성")
workflow.add_edge("개선 코드 생성", END)

# 컴파일
app = workflow.compile()

# ✅ 실행 예시
input_code = """
def add(a,b):
 return a+b
"""

result = app.invoke({
    "code": input_code,
    "style_feedback": "",
    "bug_feedback": "",
    "optimize_feedback": "",
    "improved_code": ""
})

# ✅ 결과 출력
from pprint import pprint
pprint(result)


{'bug_feedback': '제공된 코드에서 `add` 함수는 두 개의 인자를 받아서 그 합을 반환하는 간단한 함수입니다. 그러나 몇 '
                 '가지 주의해야 할 점과 버그 가능성을 지적할 수 있습니다:\n'
                 '\n'
                 '1. **타입 검증 부족**: 현재 함수는 인자 `a`와 `b`의 타입을 검증하지 않습니다. 만약 두 인자가 '
                 '숫자가 아닌 다른 타입(예: 문자열, 리스트 등)이라면, `TypeError`가 발생할 수 있습니다. 이를 '
                 '방지하기 위해 입력 타입을 확인하고 적절한 예외 처리를 추가하는 것이 좋습니다.\n'
                 '\n'
                 '   ```python\n'
                 '   def add(a, b):\n'
                 '       if not isinstance(a, (int, float)) or not '
                 'isinstance(b, (int, float)):\n'
                 '           raise TypeError("Both arguments must be int or '
                 'float")\n'
                 '       return a + b\n'
                 '   ```\n'
                 '\n'
                 '2. **정수와 부동 소수점의 혼합**: 정수와 부동 소수점 숫자를 더할 때, 결과는 부동 소수점 숫자가 '
                 '됩니다. 이로 인해 예상치 못한 결과가 발생할 수 있습니다. 예를 들어, 정수 연산을 원할 경우, 이를 '
                 '명시적으로 처리해야 할 수 있습니다.\n'
       